In [ ]:
#resampling, finding the subjects that aren't included in matlab, making mean image, making efields and smoothing, gdnf

In [1]:
from nilearn import image
from nilearn.image import load_img
from nilearn.image import math_img
from nilearn.image import resample_to_img
from nilearn.image import smooth_img
import numpy as np
import nibabel as nib
import pandas as pd
import os, glob
import shutil

In [26]:
#resample to mni: interpolation nearest!!
mni = load_img('/Users/jp1590/Documents/MATLAB/leaddbs/templates/space/MNI152NLin2009bAsym/t1.nii')
files = os.listdir('/Volumes/Cingulate/Sweetspot_Seg/Sweetspot_Seg_All_Flipped/Zone1/All_Resampled')
for i in files:
    if i.startswith('sub-3') or i.startswith('sub-4'):
        sub = load_img(f'/Volumes/Cingulate/Sweetspot_Seg/Sweetspot_Seg_All_Flipped/Zone1/All_Resampled/{i}')
        sub_re = resample_to_img(sub,mni,interpolation='nearest')
        nib.save(sub_re,f'/Volumes/Cingulate/Sweetspot_Seg/Sweetspot_Seg_All_Flipped/Zone1/cing-sct-res/{i}')

In [11]:
#determine which subjects are included in the python n-map and not matlab n-map
files = os.listdir('/Volumes/Cingulate/Sweetspot_Seg/Sweetspot_Seg_All_Flipped/Bilat/Binary/z1-2')
mean_diff = load_img('/Volumes/Cingulate/Sweetspot_Seg/Outputs/binary_zones1-2/n-map_difference.nii.gz')
ybocs_df = pd.read_csv('/Volumes/Cingulate/Sweetspot_Seg/All_YBOCS_Flip.csv')
nosignif = []
ids = []
for i in files:
    if i.startswith('sub'):
        sub = load_img(f'/Volumes/Cingulate/Sweetspot_Seg/Sweetspot_Seg_All_Flipped/Bilat/Binary/z1-2/{i}')
        mask = math_img('img1*img2',img1=sub,img2=mean_diff)
        masked_sub = mask.get_fdata()
#         nib.save(mask,f'/Volumes/Cingulate/Sweetspot_Seg/Sweetspot_Seg_All_Flipped/mask/{i}')
#         masked_sub = load_img(f'/Volumes/Cingulate/Sweetspot_Seg/Sweetspot_Seg_All_Flipped/mask/{i}').get_fdata()
        values = np.sum(masked_sub)
        nosignif.append(values)
        ids.append(i)

#export to dataframe    
nosignif_df = pd.DataFrame(
    {'ID':ids,
    'Volume_Overlap':nosignif
    })

nosignif_combined_df = nosignif_df.merge(ybocs_df,how='left',left_on=['ID'],right_on=['ID'])
nosignif_combined_df.to_csv('/Volumes/Cingulate/Sweetspot_Seg/Outputs/n-image_overlap.csv')

#find which subjects have overlap from the csv and put them in new folder
non_zero = nosignif_combined_df.loc[nosignif_combined_df['Volume_Overlap']!=0,:]

for i in non_zero['ID']:
    shutil.copyfile(f'/Volumes/Cingulate/Sweetspot_Seg/Sweetspot_Seg_All_Flipped/Bilat/Binary/z1-2/{i}',f'/Volumes/Cingulate/Sweetspot_Seg/Sweetspot_Seg_All_Flipped/outliers/{i}')

In [132]:
#replace binary segmentations with ybocs score
df = pd.read_csv('/Volumes/Cingulate/Sweetspot_Seg/mean-image-ybocs.csv')
df.dropna(subset=['Relative'], inplace=True)
df.dropna(subset=['ID'], inplace=True)
for i in df.index:
    sub_id = (df['ID'][i])
    sub = load_img(f'/Volumes/Cingulate/Sweetspot_Seg/Sweetspot_Seg_All_Flipped/Zone1/All_Resampled/{sub_id}')
    relative = df['Relative'][i] #change relative here with value to be put in the binary segmentations
    threshold = math_img(f'np.where(img1==1,{relative},0)',img1=sub)
    nib.save(threshold,f'/Volumes/Cingulate/Sweetspot_Seg/Sweetspot_Seg_All_Flipped/Zone1/All_YBOCS_Value/{sub_id}')

In [94]:
#divide n-map and ybocs to get mean image
n = load_img('/Volumes/Cingulate/Sweetspot_Seg/Outputs/binary_zone1/n-map_zone1.nii.gz')
y = load_img('/Volumes/Cingulate/Sweetspot_Seg/Outputs/binary_zone1/ybocs_sum.nii.gz')
yb = math_img('img1/img2',img1=y,img2=n)
nib.save(yb,'/Volumes/Cingulate/Sweetspot_Seg/Outputs/binary_zone1/mean_image.nii.gz')

<string>:1: RuntimeWarning: divide by zero encountered in divide
<string>:1: RuntimeWarning: invalid value encountered in divide


In [51]:
#replaces values in a nifti file
def replace(path):
    sub = load_img(f'/Volumes/Cingulate/Sweetspot_Seg/Sweetspot_Seg_All_Flipped/Bilat/Weighted_80-100/z1-2/{i}')
    sub_array = sub.get_fdata()
    new_data = np.where(sub_array == 80, 10, sub_array) #change here values to replace
    new_data = new_data.astype(np.int16)
    new_img = nib.Nifti1Image(new_data, sub.affine, sub.header)
    nib.save(new_img,f'/Volumes/Cingulate/Sweetspot_Seg/Sweetspot_Seg_All_Flipped/Bilat/Weighted_10-100/z1-2/{i}')
    return new_img
    
path = '/Volumes/Cingulate/Sweetspot_Seg/Sweetspot_Seg_All_Flipped/Bilat/Weighted_80-100/z1-2'

for i in os.listdir(path):
    if i.endswith('nii.gz'):
        replace(i)

In [2]:
#smooth segmentations gaussianly
weighted = os.listdir('/Volumes/Cingulate/Sweetspot_Seg/Sweetspot_Seg_All_Flipped/Weighted_10-100/z1-2')
for i in weighted:
    if i.endswith('.nii.gz'):
        sub = load_img(f'/Volumes/Cingulate/Sweetspot_Seg/Sweetspot_Seg_All_Flipped/Weighted_10-100/z1-2/{i}')
        gaussian = smooth_img(sub,fwhm=4) #change threshold here
        nib.save(gaussian,f'/Volumes/Cingulate/Sweetspot_Seg/Sweetspot_Seg_All_Flipped/Weighted_10-100/Efields_Unmasked_4/{i[:-18]}_smooth_bilat_flip.nii')

In [3]:
#remove excess in the efields by masking with the binary segmentations
binary = os.listdir('/Volumes/Cingulate/Sweetspot_Seg/Sweetspot_Seg_All_Flipped/Binary/z1-2')
for i in binary:
    if i.endswith('.nii'):
        bina = load_img(f'/Volumes/Cingulate/Sweetspot_Seg/Sweetspot_Seg_All_Flipped/Binary/z1-2/{i}')
        efield = load_img(f'/Volumes/Cingulate/Sweetspot_Seg/Sweetspot_Seg_All_Flipped/Weighted_10-100/Efields_Unmasked_4/{i[:-15]}_smooth_bilat_flip.nii')
        mask = math_img('img1*img2',img1=bina,img2=efield)
        nib.save(mask,f'/Volumes/Cingulate/Sweetspot_Seg/Sweetspot_Seg_All_Flipped/Weighted_10-100/Efields_Masked_4/{i[:-15]}_smooth_bilat_flip.nii')

In [82]:
#gdnf one patient
df = pd.DataFrame(columns=['ID','T1_or_Post','Inf_Date','Date_of_Image','Time_Since_Inf','path'])
for i in os.listdir('/Volumes/Cingulate/GDNF/GD06_nii'):
    if i.endswith('1_post.nii.gz'):
        t1_or_post_value = 'post'
    elif i.endswith('T1.nii.gz'):
        t1_or_post_value = 'T1'
    else:
        t1_or_post_value = 'Infusion'

    df.loc[len(df)] = {
        'ID': 'GD06',
        'path': f'/Volumes/Cingulate/GDNF/GD06_nii/{i}',
        'Inf_Date': '20150729',
        'Date_of_Image': int(i[0:8]),
        'Time_Since_Inf': f'{int(i[0:4])-2015}y {int(i[5:6])-7}m',
        'T1_or_Post': t1_or_post_value
    }

In [97]:
#gdnf all patients
df = pd.DataFrame(columns=['ID', 'Inf_Date', 'Date_of_Image', 'Time_Since_Inf', 'T1_or_Post','path'])

path = '/Volumes/Cingulate/GDNF'
inf_date = None

for folder in os.listdir(path):
    folder_path = os.path.join(path, folder)
    if os.path.isdir(folder_path):  # Check if it's a subfolder
        for i in os.listdir(folder_path):
            if i.endswith('Infusion') or i.endswith('gz'):
                if i.endswith('Infusion'):
                    inf_date = i[0:8]
                if i.endswith('1_post.nii.gz'):
                    t1_or_post_value = 'post'
                elif i.endswith('T1.nii.gz'):
                    t1_or_post_value = 'T1'
                elif i.endswith('Infusion'):
                    t1_or_post_value = 'Infusion'
                    inf_date = i[0:8]
                else:
                    t1_or_post_value = 'None'

                df.loc[len(df)] = {
                    'ID': folder[:-4],
                    'path': os.path.join(folder_path, i),
                    'Inf_Date': inf_date,
                    'Date_of_Image': int(i[0:8]),
                    'Time_Since_Inf': f'{int(i[0:4])-2015}y {int(i[5:6])-7}m', #need to update in csv with correct subtraction
                    'T1_or_Post': t1_or_post_value
                }

In [111]:
#fix up time since infusion column
ddf = pd.read_csv('/Volumes/Cingulate/GDNF/gdnf.csv')
ddf['Time_Since_Inf'] = ddf.apply(lambda row: f'{(row["Date_of_Image"] // 10000) - (row["Inf_Date"] // 10000)}y {(row["Date_of_Image"] // 100 % 100) - (row["Inf_Date"] // 100 % 100)}m', axis=1)

In [7]:
#get the volume of each lesion
vol = []
ids = []
for i in os.listdir('/Volumes/Empty/Sweetspot_Seg/Sweetspot_Seg_All_Flipped/Binary/z1-2'):
    if i.endswith('.nii'):
        sub = load_img(f'/Volumes/Empty/Sweetspot_Seg/Sweetspot_Seg_All_Flipped/Binary/z1-2/{i}').get_fdata()
        sub_vol = np.sum(sub)
        vol.append(sub_vol)
        ids.append(i)
        
df = pd.DataFrame(
    {'ID':ids,
    'Volume':vol
})
df.to_csv('/Volumes/Empty/Sweetspot_Seg/binary_vols.csv')

In [4]:
df = pd.read_csv('/Volumes/Empty/Sweetspot_Seg/All_YBOCS_Flip_No_Volume.csv')
for i in df['ID']:
    df['ID'] = df['ID'].replace(i,f'/Volumes/Empty/Sweetspot_Seg/Sweetspot_Seg_All_Flipped/Weighted_50-100/Efields_Masked_1.5/{i}')